In [2]:
# Install packages
# !conda install -c conda-forge plyvel


In [3]:
# import readme.md file (project overview) and display the markdown file here

from IPython.display import display, Markdown

with open('README.md', 'r') as f:
    content = f.read()

display(Markdown(content))


# ChatGPT local analysis in Jupyter Notebook 🤖

In this notebook I deliver a foundation on how to analyse the personal ChatGPT conversation history. 

After a few months of using ChatGPT, I have collected a large amount of data and want to reflect on old conversations and spark ideas on how to use it in the future.

Using the Superpower ChatGPT extension for Chrome you can automatically sync your conversations for offline usage. Since all your conversations are now stored locally, you can analyse the local database to get insights into your conversations.

## Demo
Feel free to check out the notebook [here](./chatgpt_analysis.ipynb): 
## Requirements

- Superpower ChatGPT extension: https://github.com/saeedezzati/superpower-chatgpt
- Python 3.8
- Streamlit


## Current state

- [x] Get data from local database to a pandas dataframe
    - [x] df_conversations
    - [x] df_messages

- [x] Streamlint Dashboard
    - [x] Basic setup
    - [x] Table with conversations
    - [x] Wordcloud of all messages
    - [x] Exclusion list in custom_stop_words.txt
    
    - [ ] Conversation overview
    - [ ] Graph network of conversations / words / topics

## Future features

- Graph network of conversations / words / topics (Can someone help me with this?) 
    - Not sure how to do this, but I think it would be cool to see how conversations are connected and how topics are connected to each other
    - https://towardsdatascience.com/how-to-deploy-interactive-pyvis-network-graphs-on-streamlit-6c401d4c99db

## Pull requests are welcome!
Feel free to contribute to this project and contribute ideas on how to analyse the data.

## Import data


### Import libraries

In [4]:
import plyvel
import os
import json
import pandas as pd


### Import data from browser database

In [5]:

# Path to the leveldb directory (change this to your path - in my case I am using Brave Browser)
leveldb_path = '~/Library/Application Support/BraveSoftware/Brave-Browser/Default/Local Extension Settings/amhmeenmapldpjdedekalnfifgnpfnkc'
leveldb_path = os.path.expanduser(leveldb_path)  # Expand the '~' symbol to the user's home directory

# Enclose the path in quotes to handle spaces
leveldb_path = f'"{leveldb_path}"'

# Get the list of files in the leveldb directory

!ls -l $leveldb_path

# Copy all files to the current directory /db

!cp -r $leveldb_path ./db

# set leveldb path to the copied directory

leveldb_path = './db/amhmeenmapldpjdedekalnfifgnpfnkc'


total 75656
-rw-------@ 1 phil  staff   7720330 May 30 17:41 003033.ldb
-rw-------@ 1 phil  staff    131157 Jun  9 16:46 006828.ldb
-rw-------@ 1 phil  staff    127701 Jun 10 09:25 007043.ldb
-rw-------@ 1 phil  staff  22356554 Jun 10 09:26 007050.log
-rw-------@ 1 phil  staff   7916084 Jun 10 09:26 007052.ldb
-rw-------@ 1 phil  staff        16 May 28 21:05 CURRENT
-rw-------@ 1 phil  staff         0 May 28 21:05 LOCK
-rw-------@ 1 phil  staff     70166 Jun 10 09:26 LOG
-rw-------@ 1 phil  staff      1774 Jun  8 18:10 LOG.old
-rw-------@ 1 phil  staff    320509 Jun 10 09:26 MANIFEST-000001


### Initiate Database

In [6]:
db = plyvel.DB(leveldb_path)

# get database info

print(db.get_property(b'leveldb.stats'))


# filter all key entries that contain chat.openai.com 

for key, value in db:
    if b'chat.openai.com' in key:
        print(f'Key: {key}, Value: {value}')
        



b'                               Compactions\nLevel  Files Size(MB) Time(sec) Read(MB) Write(MB)\n--------------------------------------------------\n  0        1        7         0        0         7\n  1        2        8         0        0         0\n  2        2        7         0        0         0\n'


### Create dataframe

In [7]:

df = pd.DataFrame(columns=['key', 'value'])

for key, value in db:
    df = pd.concat([df, pd.DataFrame({'key': [key], 'value': [value]})], ignore_index=True)
    # df = df.append({'key': key, 'value': value}, ignore_index=True)



### Extract conversations

In [8]:
# read the value of key "conversations"  as json in a separate dataframe

df_conversations = pd.DataFrame(json.loads(df[df['key'] == b'conversations']['value'].values[0]))
# with column / lines transposed
df_conversations = df_conversations.T
# also export as text file
df_conversations.to_csv('df_conversations.txt', sep='\t', index=False)

# # add a column for the key value length
df['key_length'] = df['value'].str.len()

df.head()

# close the database
db.close()

### Extract messages

In [9]:

# drop all rows of df_conversations with empty mapping
df_conversations = df_conversations[df_conversations['mapping'].str.len() > 0]

# Create a new DataFrame for messages
df_messages = pd.DataFrame(columns=['id','conversation_id', 'title', 'message', 'role', 'create_time'])

# Iterate over the conversations and messages
for idx, conversation in df_conversations.iterrows():
    mapping = conversation['mapping']
    title = conversation['title']
    conversation_id = conversation['id']
    
    # Iterate over the messages in the mapping
    for message_id, message_data in mapping.items():
        if 'message' in message_data and message_data['message'] is not None:
            message = message_data['message']
            
            # Get the message text
            message_text = ''
            if 'content' in message and 'parts' in message['content']:
                message_parts = message['content']['parts']
                if len(message_parts) > 0:
                    message_text = message_parts[0]
            
            # Get the author role
            role = ''
            if 'author' in message and 'role' in message['author']:
                role = message['author']['role']
            
            # Get the create time
            create_time = ''
            if 'create_time' in message:
                create_time = message['create_time']
            
            # Append the message data to the DataFrame
            # df_messages = df_messages.append({
            #     'id': message_id,
            #     'title': title,
            #     'message': message_text,
            #     'role': role,
            #     'create_time': create_time
            # }, ignore_index=True)

            df_messages = pd.concat([df_messages, pd.DataFrame({
                'id': [message_id],
                'conversation_id': [conversation_id],
                'title': [title],
                'message': [message_text],
                'role': [role],
                'create_time': [create_time]
                })], ignore_index=True)
            


# Close the LevelDB database
db.close()

# Display the first few rows of the messages DataFrame
print(df_messages.head())




                                     id                       conversation_id  \
0  430741cb-bd4b-44bd-b38a-fffeb45a3b49  000250bf-4b4b-47de-9bb9-2425ebccf8c0   
1  72ed73e5-3856-4b72-9700-23dc8acd0015  000250bf-4b4b-47de-9bb9-2425ebccf8c0   
2  af30cc11-ce74-475a-8a21-aa9d375ea1e7  000250bf-4b4b-47de-9bb9-2425ebccf8c0   
3  eea89b97-3e61-424c-9f92-c57d7706973f  000250bf-4b4b-47de-9bb9-2425ebccf8c0   
4  f248e179-f8fd-4184-8163-20a44ab381fa  000250bf-4b4b-47de-9bb9-2425ebccf8c0   

      title                                            message       role  \
0  New chat  Any ideas for a cycling-related data-driven gr...       user   
1  New chat                                                        system   
2  New chat  Any ideas for a data-driven graduation project...       user   
3  New chat  Here are a few ideas for a cycling-related dat...  assistant   
4  New chat  Here are a few ideas for a data-driven graduat...  assistant   

         create_time  
0  1671022256.202531  
1  1

## Data Cleaning



In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

# add more stop words in english and german
stop_words = stopwords.words('english')
# also exclude code words like < >, the, to, and =, a, in , of, for [ ] ( ) {} etc.
code_words = ['<', '>', 'the', 'to', 'and', '=', 'a', 'in', 'of', 'for', '[', ']', '(', ')', '{', '}', 'const', 'import', 'script', 'button', 'await', 'null', 'code', 'div', 'und', 'px', 'data', 'file', 'die', 'return', 'image', 'user', 'der', 'use', 'error', 'value', 'new', 'color', 'zu', 'create', 'using', 'component', 'add', 'false', 'object', 'template', 'name', 'da', 'also', 'app', 'example', 'span', 'für', 'width', 'mit', 'type', 'content', 'label', 'method', 'display', 'feedback', 'bike', 'rating', 'style', 'location', 'e', 'backgroundcolor', 'try', 'height', 'center', 'button', 'title', 'div', 'px', 'color', 'null', 'file', 'template', 'da', 'false', 'value', 'script', 'span', 'error', 'backgroundcolor', 'e', 'button', 'sie', 'ist', 'true', 'cycling', 'make', 'eine', 'class', 'default', 'auf', 'could', 'von', 'heres', 'like', 'name', 'function', 'used', 'i', 'need', 'async', 'based', 'label', 'data', 'advice', 'style', 'center', 'submission', 'infrastructure', 'set', 'model', 'property', 'width', 'type', 'id', 'ein', 'report', 'div', 'px', 'color', 'null', 'file', 'template', 'da', 'false', 'value', 'script', 'span', 'error', 'backgroundcolor', 'e', 'button', 'heres', 'name', 'i', 'label', 'data', 'style', 'center', 'width', 'type', 'index', 'text', 'vue', 'map', 'get', 'den', 'title', 'image', 'user', 'issue', 'array', 'f', 'column', 'comment', 'element', 'true', 'list', 'display', 'p', 'sure', 'im', 'result', 'height']

lemmatizer = WordNetLemmatizer()

df_messages['cleaned_text'] = df_messages['message'].str.lower()
# strip out words from stop_words and code_words from cleaned_text column
df_messages['cleaned_text'] = df_messages['cleaned_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words + code_words)]))
df_messages['cleaned_text'] = df_messages['cleaned_text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
df_messages['cleaned_text'] = df_messages['cleaned_text'].str.replace('[^\w\s]','')
df_messages['cleaned_text'] = df_messages['cleaned_text'].str.replace('[\d+]','')

df_messages.head()

# based on df_messages['cleaned_text'] column create word frequency count 

from collections import Counter

# Create a list of all words in the messages
all_words = []

for idx, row in df_messages.iterrows():
    all_words.extend(row['cleaned_text'].split())

# Create a word frequency counter

word_freq = Counter(all_words)

# Display the 10 most common words

print(word_freq.most_common(10))

# Create a list of all words in the messages

all_words = []

# save the messages dataframe as txt

df_messages.to_csv('df_messages.txt', sep='\t', index=False)





[nltk_data] Downloading package stopwords to /Users/phil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/phil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/var/folders/wc/1fgpcwpx4rzfbbc7shb_wy8h0000gn/T/ipykernel_57513/1433106567.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df_messages['cleaned_text'] = df_messages['cleaned_text'].str.replace('[^\w\s]','')
/var/folders/wc/1fgpcwpx4rzfbbc7shb_wy8h0000gn/T/ipykernel_57513/1433106567.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df_messages['cleaned_text'] = df_messages['cleaned_text'].str.replace('[\d+]','')


[('div', 11351), ('px', 7869), ('color', 3109), ('null', 2996), ('file', 2651), ('template', 2567), ('da', 2432), ('false', 2331), ('value', 2319), ('script', 2258)]


In [11]:
# install pyvis and jinja2
# !pip install pyvis
# !pip install jinja2


# Dashboard

Initialize the streamlint dashboard

### Installations

In [12]:
# install streamlit
# !pip install streamlit

In [13]:
# install worldcloud
# !pip install wordcloud

In [14]:
# install networkx
# %pip install networkx

### Initialize Dashboard

In [15]:
# run streamlit app

code = """

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# read custom_stop_words from txt file

custom_stop_words = []
with open('custom_stop_words.txt', 'r') as f:
    custom_stop_words = f.read().splitlines()


# Read the chat logs data into a DataFrame
df_messages = pd.read_csv('df_messages.txt', sep='	')
df_messages['cleaned_text'] = df_messages['cleaned_text'].astype(str)  # Convert 'cleaned_text' column to string type
# filter out code words in cleaned_text column 
df_messages['cleaned_text'] = df_messages['cleaned_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in custom_stop_words]))

# Process the chat messages to extract words
all_words = ' '.join(df_messages['cleaned_text']).lower().split()
all_words = [word for word in all_words if word not in custom_stop_words]
word_counts = Counter(all_words)
most_common_words = word_counts.most_common(200)  # Change the number as per your requirement

# Create a DataFrame for the most common words
df_common_words = pd.DataFrame(most_common_words, columns=['Word', 'Count'])

# Display the most common words in a bar chart
fig, ax = plt.subplots()
df_common_words.plot.bar(x='Word', y='Count', ax=ax)
plt.xlabel('Word')
plt.ylabel('Count')
plt.title('Most Common Words in Chat Logs')
plt.xticks(rotation=45)
st.pyplot(fig)

# Display the raw data of the most common words
st.write(df_common_words)

# Display the raw data of the chat logs
st.write(df_messages)

# display a word cloud
from wordcloud import WordCloud

# Create a word cloud

wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=200).generate(' '.join(df_messages['cleaned_text']))

# Display the generated image:
# the matplotlib way:

fig, ax = plt.subplots()
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()
st.pyplot(fig)
"""

In [16]:
# Write the code to the dashboard.py file
with open('dashboard.py', 'w') as f:
    f.write(code)

In [17]:
# run streamlit app dashboard.py
!streamlit run dashboard.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.24.144.160:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
^C
  Stopping...


In [ ]:

# close the database
db.close()